In [ ]:
execfile('d:/measuring/analysis/scripts/setup_analysis.py')
%matplotlib inline


In [ ]:
def fit_lifetime(t,cts,g_tau=10.5,t_start = 200,t_stop=300,do_plot=False,name='',fit_double=False):
    g_a = 0
    g_A = 600
    g_tau = g_tau
    g_tau2 = 18.5
    fixed=[]
    i_start = np.argmin(abs(t-t_start))
    i_stop = np.argmin(abs(t-t_stop))
    if fit_double:
        p0, fitfunc, fitfunc_str = common.fit_double_exp_decay_with_offset(g_a,g_A,g_tau,g_A,g_tau)
    else:
        p0, fitfunc, fitfunc_str = common.fit_exp_decay_with_offset(g_a,g_A,g_tau)
    fit_res=fit.fit1d(t[i_start:i_stop]-t[i_start],cts[i_start:i_stop], fitmethod=None,p0=p0,fixed=fixed, fitfunc=fitfunc, fitfunc_str=fitfunc_str,do_print=False)
    if fit_double:
        try:
            tau=fit_res['params_dict']['tau']
            u_tau =fit_res['error_dict']['tau']
            tau2= fit_res['params_dict']['tau2']
            u_tau2 = fit_res['error_dict']['tau2']
        except:
            tau=0
            u_tau=0
            tau2=0
            u_tau2=0
    else:
        try:
            tau= fit_res['params_dict']['tau']
            u_tau = fit_res['error_dict']['tau']
        except:
            tau=0
            u_tau=0
#     print fit_res['fitfunc'](t)
    if do_plot:
        ax = plot.plot_fit1d(fit_res,print_info=False, ret='ax')
        ax.set_title(folder+'_'+name)
        ax.set_yscale('log')
        ax.text((max(t[i_start:i_stop])-t[i_start])/2,max(cts[i_start:i_stop])/2+1,'tau = %.1f +- %.1f ns'%(tau,u_tau),color='c')
        if fit_double:
            ax.text((max(t[i_start:i_stop])-t[i_start])/2,max(cts[i_start:i_stop])/3+1,'tau2 = %.1f +- %.1f ns'%(tau2,u_tau2),color='c')

        ax.set_ylim(min(cts[i_start:i_stop]),max(cts[i_start:i_stop])*1.1)
        ax.set_xlabel('time (ns)')
        ax.set_ylabel('cts')
#         plt.savefig(os.path.join(folder,'fit_lifetime_'+name+'.png'))
        plt.show()
        plt.close()
    if fit_double:
        return tau,u_tau,tau2,u_tau2
    return tau,u_tau

In [ ]:
folder= tb.latest_data(contains='Measurement_GiuseppeII_ZPL')
print folder
a = m2.M2Analysis(folder)

lt_grp = a.g['qutau']
t_ZPL_raw = lt_grp['t_ZPL'].value
t_PSB_raw = lt_grp['t_PSB'].value
cts_ZPL_raw =lt_grp['cts_ZPL'].value
cts_PSB_raw =lt_grp['cts_PSB'].value
tot_msmt_times = lt_grp['tot_msmt_times'].value
tot_syncs = lt_grp['tot_nrs_syncs'].value#np.ones(len(sweep_points))*1.e6*10#

print np.shape(t_ZPL_raw)
print np.shape(t_PSB_raw)
print np.shape(cts_ZPL_raw)

print tot_msmt_times
print sum(cts_ZPL_raw)
# print float(sum(cts_ZPL_raw)/float(tot_syncs))
print 'cr rate sync', tot_syncs/tot_msmt_times, 'Hz'

In [ ]:
plt.plot(t_ZPL_raw, cts_ZPL_raw)

In [ ]:
lifetime= False
nr_bins =a.g.attrs['nr_bins']#9#
print nr_bins
sweep_points = a.g.attrs['sweep_points']
nr_sweep_points=4#len(sweep_points)
print np.shape(t_PSB_raw)
print np.shape(cts_PSB_raw)
print nr_sweep_points*nr_bins

nr_extra_bins = 20
qutau_tb=8.1e-11
qutau_counter_bins=2
total_binsize=qutau_tb*qutau_counter_bins*nr_extra_bins
print 'binsize:', total_binsize*1.e9, 'ns'


t_ZPL = np.reshape(t_ZPL_raw, (nr_sweep_points,nr_bins))
cts_ZPL = np.reshape(cts_ZPL_raw, (nr_sweep_points,nr_bins))

t_ZPLCut = t_ZPL[:,len(t_ZPL[0,:])%nr_extra_bins:]
cts_ZPLCut = cts_ZPL[:,len(t_ZPL[0,:])%nr_extra_bins:]

t_ZPLBinned = np.mean(t_ZPLCut.reshape(nr_sweep_points,-1,nr_extra_bins), axis=2)
cts_ZPLBinned =np.mean(cts_ZPLCut.reshape(nr_sweep_points,nr_bins/nr_extra_bins, nr_extra_bins), axis=2)

t_ZPL = t_ZPLBinned
cts_ZPL=cts_ZPLBinned
t_PSB = np.reshape(t_PSB_raw, (nr_sweep_points,nr_bins))
cts_PSB = np.reshape(cts_PSB_raw, (nr_sweep_points,nr_bins))

#print cts_ZPL[i]
# print len(cts_ZPL[1])

if lifetime:
    fig,ax = plt.subplots()
    ax.plot(t_PSB[0],cts_PSB[0])
    ax.set_yscale('log')
# ax.set_xlim(0,270)
# ax.set_ylim(1000,max(cts_PSB[0]))

for i in np.arange(nr_sweep_points):
    print 'total cts ZPL', sum(cts_ZPL_raw)
    print 'cr rate sync', tot_syncs/tot_msmt_times, 'Hz'
    fig,ax = plt.subplots()
    ax.plot(t_ZPL[i],cts_ZPL[i])
#     ax.set_xlim(0,100)
    ax.set_xlabel('ns')
    ax.set_ylabel('counts')
    plt.show()

    # fig,ax = plt.subplots()
# ax.plot(t_ZPLBinned[0],cts_ZPLBinned[0])
# # ax.set_yscale('log')
# plt.show()


In [ ]:
tau_ZPL=np.zeros(len(sweep_points))
tau2_ZPL=np.zeros(len(sweep_points))
tau_PSB=np.zeros(len(sweep_points))
tau2_PSB=np.zeros(len(sweep_points))
u_tau2_ZPL=np.zeros(len(sweep_points))
u_tau_ZPL=np.zeros(len(sweep_points))
u_tau_PSB=np.zeros(len(sweep_points))
u_tau2_PSB=np.zeros(len(sweep_points))
countspershot_ZPL=np.zeros(len(sweep_points))
countspershot_PSB=np.zeros(len(sweep_points))
countrate_ZPL=np.zeros(len(sweep_points))
countrate_PSB=np.zeros(len(sweep_points))


for i,V in enumerate(sweep_points):
    #,tau2_ZPL[i],u_tau2_ZPL[i] 
    tau_ZPL[i],u_tau_ZPL[i]= fit_lifetime(t_ZPL[i],cts_ZPL[i],t_start=226,t_stop=226+20, g_tau=11.7,do_plot=True,name='ZPL_%d'%i,fit_double=False)
    tau_PSB[i],u_tau_PSB[i] = fit_lifetime(t_PSB[i],cts_PSB[i],t_start = 200,t_stop=240,g_tau=11.7,do_plot=True,name='PSB_%d'%i,fit_double=False)
    countspershot_ZPL[i] = np.sum(cts_ZPL[i])/float(tot_syncs[i])
    countspershot_PSB[i] = np.sum(cts_PSB[i])/float(tot_syncs[i])
    countrate_ZPL[i] = np.sum(cts_ZPL[i])/tot_msmt_times[i]
    countrate_PSB[i] = np.sum(cts_PSB[i])/tot_msmt_times[i]
  

In [ ]:
  
print countspershot_ZPL
fig,ax = plt.subplots()
ax.errorbar(sweep_points,tau_PSB,yerr=u_tau_PSB,fmt='bo',label='measured in PSB')
# ax.errorbar(sweep_points,tau2_PSB,yerr=u_tau2_PSB,fmt='bo',label='measured in PSB')
ax.errorbar(sweep_points,tau_ZPL,yerr=u_tau_ZPL,fmt='ro',label='measured in ZPL')
ax.legend()
ax.set_title(folder)
ax.set_xlabel('voltage (V)',fontsize=14)
ax.set_ylabel('lifetime',fontsize=14)
plt.show()
plt.savefig(os.path.join(folder,'all_lifetimes.png'))

fig,ax = plt.subplots()
ax2=ax.twinx()
ax.plot(sweep_points,countspershot_ZPL*1000,'co',label='measured in ZPL')
ax2.plot(sweep_points,countspershot_PSB*1000,'mo',label='measured in PSB')
ax.set_title(folder)
ax.set_xlabel('voltage (V)',fontsize=14)
ax.set_ylabel('x 10^-3 counts per pulse, ZPL',color='c',fontsize=14)
ax2.set_ylabel('x 10^-3 counts per pulse, PSB', color='m',fontsize=14)
plt.show()
plt.savefig(os.path.join(folder,'counts_per_shot.png'))

fig,ax = plt.subplots()
ax2=ax.twinx()
ax.plot(sweep_points,countrate_ZPL,'co',label='measured in ZPL')
ax2.plot(sweep_points,countrate_PSB,'mo',label='measured in PSB')
# ax.legend()
ax.set_title(folder)
ax.set_xlabel('voltage (V)',fontsize=14)
ax.set_ylabel('counts in ZPL (Hz)',color='c',fontsize=14)
ax2.set_ylabel('counts in PSB (Hz)',color='m',fontsize=14)
plt.show()
plt.savefig(os.path.join(folder,'countrate.png'))

In [ ]:
g_a = 0
g_A = 600
g_tau = 12
t_ZPL_start = 225
i_ZPL_start = np.argmin(abs(t_ZPL-t_ZPL_start))
t_PSB_start = 200
i_PSB_start = np.argmin(abs(t_PSB-t_PSB_start))
print i_ZPL_start, i_PSB_start

p0, fitfunc, fitfunc_str = common.fit_exp_decay_with_offset(g_a,g_A,g_tau)
fit_res=fit.fit1d(t_ZPL[i_ZPL_start:]-t_ZPL[i_ZPL_start],cts_ZPL[i_ZPL_start:], fitmethod=None,p0=p0, fitfunc=fitfunc, fitfunc_str=fitfunc_str,do_print=True)
ax = plot.plot_fit1d(fit_res,print_info=False, ret='ax')
tau= fit_res['params_dict']['tau']
u_tau = fit_res['error_dict']['tau']
ax.text(300,max(cts_ZPL)/2,'tau ZPL = %.1f +- %.1f ns'%(tau,u_tau),color='c')


p0, fitfunc, fitfunc_str = common.fit_exp_decay_with_offset(g_a,g_A,g_tau)
fit_res=fit.fit1d(t_psb[i_PSB_start:]-t_PSB[t_PSB_start],cts_PSB[i_PSB_start:], fitmethod=None,p0=p0, fitfunc=fitfunc, fitfunc_str=fitfunc_str,do_print=True)
ax = plot.plot_fit1d(fit_res,print_info=False,ax=ax, ret='ax')
tau= fit_res['params_dict']['tau']
u_tau = fit_res['error_dict']['tau']
ax.text(300,max(cts_PSB)/2,'tau PSB = %.1f +- %.1f ns'%(tau,u_tau),color='m')

ax.set_title(folder)

ax.set_yscale('log')
ax.set_ylim(1,max(y_psb)*1.1)
ax.set_xlabel('time (ns)')
ax.set_ylabel('cts')

In [ ]:
# Counts only

In [ ]:

folder= tb.latest_data(contains='Measurement_GiuseppeII_ZPL_test')
print folder
a = m2.M2Analysis(folder)

lt_grp = a.adwingrp('lifetime')
t_ZPL_raw = lt_grp['t_ZPL'].value
t_PSB_raw = lt_grp['t_PSB'].value
cts_ZPL_raw =lt_grp['cts_ZPL'].value
cts_PSB_raw =lt_grp['cts_PSB'].value
tot_msmt_times = lt_grp['tot_msmt_times'].value
tot_syncs = lt_grp['tot_nrs_syncs'].value#np.ones(len(sweep_points))*1.e6*10#

print np.shape(t_ZPL)
print tot_syncs

In [ ]:
%time 10 - math.fmod(10,0.24)

In [ ]:
print math.fmod((8*0.3),0.3)

print np.fmod((8*0.3),0.3)

print (8*0.3) %0.3